In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
start_time = '20070101'
end_time = '20211101'
raw_data_path = r'/Users/daomeidan/Desktop/Machine_Learning/data/data1'
raw_data_path2 = r'/Users/daomeidan/Desktop/Machine_Learning/data/data2'
raw_data_path3 = r'/Users/daomeidan/Desktop/Machine_Learning/data/data3'

In [4]:
raw_data_path

'/Users/daomeidan/Desktop/Machine_Learning/data/data1'

In [5]:
import tushare as ts
ts.set_token('67d73bf318053cb9afd02a7d9996b16b882c8699323d03a945248d11')
pro = ts.pro_api()

In [6]:
def reduce_mem_usage(df, verbose=True): # 天池大赛，压缩数据量
    start_mem = df.memory_usage().sum() / 1024 ** 2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else: 
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem)/start_mem))
    return df
            

In [7]:
#查询所有正常上市交易的股票列表
data = pro.stock_basic(exchange='', list_status='L', 
                      fields = 'ts_code,name,area,industry,market,list_status,list_date,delist_date')

In [23]:
data

,ts_code,name,area,industry,market,list_status,list_date,delist_date
0,000001.SZ,平安银行,深圳,银行,主板,L,19910403,None
1,000002.SZ,万科A,深圳,全国地产,主板,L,19910129,None
2,000004.SZ,国华网安,深圳,软件服务,主板,L,19910114,None
3,000005.SZ,ST星源,深圳,环境保护,主板,L,19901210,None
4,000006.SZ,深振业A,深圳,区域地产,主板,L,19920427,None
...,...,...,...,...,...,...,...,...
4623,871553.BJ,凯腾精工,None,None,北交所,L,20210806,None
4624,871642.BJ,通易航天,None,None,北交所,L,20210816,None
4625,871981.BJ,晶赛科技,None,None,北交所,L,20211115,None
4626,872925.BJ,锦好医疗,None,None,北交所,L,20211025,None


In [31]:
data.isnull().sum()

ts_code           0
name              0
area             81
industry         81
market            0
list_status       0
list_date         0
delist_date    4628
dtype: int64

In [13]:
# data.to_csv(raw_data_path+'stock_basic.csv', index=False)

In [14]:
df = pro.stock_company(exchange='', fields='ts_code,reg_capital,city,employees')

In [27]:
df
# 上市公司基本信息：
# reg_capital 注册资本

In [16]:
# df.to_csv(raw_data_path+'register_capital.csv', index=False)

In [18]:
# 日线行情，通用行情接口 使用的是复权涨跌幅
# ts_code str 股票代码
# trade_date str 交易日期
# open, high, low, close float 
# pre_close float 昨日收盘价
# change float 涨跌额
# pct_chg float 涨跌幅（未复权，如果复权需要使用通用行情接口）
# vol float成交量（手）
# amount float成交额（千元）

In [19]:
df = ts.pro_bar(ts_code='000001.SZ', adj='qfq', start_date='20210101', end_date='20211101')
# 前复权的例子

In [28]:
# df

In [29]:
df = ts.pro_bar(ts_code='000001.SZ', adj='hfq', start_date='20210101', end_date='20211101')
# df # 后复权

In [6]:
import time

In [30]:
# 前复权数据下载

n = 0
f = ts.pro_bar
adj = 'qfq'
for i in data.ts_code.values:
    print(i)
    if n == 0:
            df1 = f(ts_code=i, start_date=start_time, adj=adj, end_date=end_time)
            n += 1
    else:
            df2 = f(ts_code=i, start_date=start_time, adj=adj, end_date=end_time)
            df1 = df1.append(df2)        
            

In [25]:
# 内存优化
# df_qfq = reduce_mem_usage(df1, verbose=True)

Memory usage after optimization is: 394.47 MB
Decreased by 52.1%


In [26]:
# 数据存储
# df_qfq.to_csv(raw_data_path + 'daily_qfq.csv', index=False)

In [ ]:
# 下面是tushare给的示例代码
# df = pro.daily(trade_date='20070101')
# df = pro.trade_cal(exchange='SSE', is_open='1', 
#                             start_date='20070101', 
#                             end_date='20211101', 
#                             fields='cal_date')

# print(df.head())

In [ ]:
# def get_daily(self, ts_code='', trade_date='', start_date='', end_date=''):
#     for _ in range(3):
#         try:
#             if trade_date:
#                 df = self.pro.daily(ts_code=ts_code, trade_date=trade_date)
#             else:
#                 df = self.pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
#         except:
#                 time.sleep(1)
#         else:
#                 return df

In [ ]:
# for date in df['cal_date'].values:
#      df = get_daily(date)

In [31]:
pro = ts.pro_api()

In [65]:
fields = ['ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv']

In [34]:
list(data.ts_code.values).index(i)

4540

In [35]:
print(i)

689009.SH


In [8]:
len(list(data.ts_code.values))

4541

In [9]:
a = list(data.ts_code.values)[:]

In [31]:
# 每日指标样例
sample = pro.daily_basic(ts_code='600000.SH, 600001.SH', start_date=start_time, end_date=end_time, fields='ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv')
sample

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
0,600000.SH,20211101,8.72,0.2212,0.4232,2.09,4.3884,4.6436,0.4655,1.3033,1.3391,6.8807,5.5046,2.935217e+06,2.935217e+06,1.534481e+06,2.559509e+07,2.559509e+07
1,600000.SH,20211029,8.94,0.1226,0.2345,1.14,4.4991,4.4320,0.4860,1.3362,1.3643,6.7114,5.3691,2.935217e+06,2.935217e+06,1.534481e+06,2.624084e+07,2.624084e+07
2,600000.SH,20211028,8.96,0.0960,0.1835,0.81,4.5091,4.4419,0.4871,1.3392,1.3673,6.6964,5.3571,2.935217e+06,2.935217e+06,1.534481e+06,2.629954e+07,2.629954e+07
3,600000.SH,20211027,8.99,0.1165,0.2229,1.02,4.5242,4.4568,0.4888,1.3437,1.3719,6.6741,5.3393,2.935217e+06,2.935217e+06,1.534481e+06,2.638760e+07,2.638760e+07
4,600000.SH,20211026,9.03,0.1048,0.2004,0.93,4.5444,4.4766,0.4909,1.3497,1.3780,6.6445,5.3156,2.935217e+06,2.935217e+06,1.534481e+06,2.650501e+07,2.650501e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4180,600001.SH,20070108,4.78,2.2474,2.2493,0.62,14.7786,18.4798,1.2358,0.6710,0.6626,2.0900,2.0900,2.791445e+05,1.423510e+05,1.422315e+05,1.334311e+06,6.804380e+05
4181,600000.SH,20070105,20.83,2.6801,3.0518,1.21,35.4621,27.0783,3.8116,6.2266,3.0698,0.5611,0.5611,4.354883e+05,1.332263e+05,1.170000e+05,9.071221e+06,2.775105e+06
4182,600001.SH,20070105,4.76,3.4519,3.4548,0.96,14.7168,18.4024,1.2306,0.6682,0.6598,2.1000,2.1000,2.791445e+05,1.423510e+05,1.422315e+05,1.328728e+06,6.775909e+05
4183,600000.SH,20070104,21.46,3.8198,4.3495,1.96,36.5347,29.5067,3.9268,6.4150,3.4047,0.5446,0.5446,4.354883e+05,1.332263e+05,1.170000e+05,9.345578e+06,2.859037e+06


In [13]:
# daily数据：每日指标
n=0
not_get_list=[]
ts_list = data.ts_code.values
ts_len = data.ts_code.values.shape[0]
ts_list

array(['000001.SZ', '000002.SZ', '000004.SZ', ..., '871981.BJ',
       '872925.BJ', '689009.SH'], dtype=object)

In [12]:
import time

In [14]:

for i in range(ts_len):
    ts_code = ts_list[i]
    df2=[]
    print(i)
    try: 
        if n == 0:
            df1 = pro.daily_basic(ts_code=ts_code, start_date=start_time, end_date=end_time, fields='ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv')
            n += 1
        else:
            df2 = pro.daily_basic(ts_code=ts_code, start_date=start_time, end_date=end_time, fields='ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv')
            df1 = df1.append(df2) 
    except:
        time.sleep(1)
        print("Cannot get：")
        print(i)
        not_get_list.append(i)

0
1
2
3
4
5
6
7
8
9
10
Cannot get：
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
Cannot get：
33
34
Cannot get：
34
35
36
37
38
39
40
41
Cannot get：
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
Cannot get：
57
58
59
Cannot get：
59
60
Cannot get：
60
61
62
63
64
65
66
Cannot get：
66
67
68
69
70
71
72
73
Cannot get：
73
74
75
76
77
78
79
Cannot get：
79
80
81
82
83
84
85
86
87
88
Cannot get：
88
89
90
91
92
93
94
Cannot get：
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
Cannot get：
110
111
112
113
114
115
Cannot get：
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
Cannot get：
139
140
141
142
143
144
145
146
147
148
149
150
151
Cannot get：
151
152
153
154
155
156
157
Cannot get：
157
158
159
Cannot get：
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
Cannot get：
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Cannot get：
20

1481
1482
1483
1484
1485
1486
1487
1488
1489
1490
1491
1492
1493
1494
Cannot get：
1494
1495
1496
1497
1498
1499
1500
1501
1502
1503
1504
1505
1506
1507
1508
1509
1510
1511
1512
1513
1514
1515
1516
1517
1518
1519
1520
1521
1522
1523
1524
1525
Cannot get：
1525
1526
1527
1528
1529
1530
1531
1532
1533
1534
1535
1536
1537
1538
1539
1540
1541
1542
1543
1544
Cannot get：
1544
1545
1546
1547
1548
1549
Cannot get：
1549
1550
1551
1552
1553
Cannot get：
1553
1554
1555
1556
1557
1558
Cannot get：
1558
1559
1560
Cannot get：
1560
1561
1562
1563
1564
1565
1566
1567
1568
1569
1570
1571
1572
1573
1574
1575
1576
1577
1578
Cannot get：
1578
1579
1580
1581
1582
1583
1584
1585
1586
1587
1588
1589
1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
Cannot get：
1623
1624
1625
1626
Cannot get：
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
Cannot get：
1640
1641
1642
164

2841
2842
2843
2844
2845
Cannot get：
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
Cannot get：
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
Cannot get：
2868
2869
2870
2871
2872
2873
2874
2875
2876
2877
2878
2879
2880
2881
2882
2883
2884
Cannot get：
2884
2885
2886
2887
2888
Cannot get：
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2898
2899
2900
2901
Cannot get：
2901
2902
2903
2904
2905
Cannot get：
2905
2906
2907
Cannot get：
2907
2908
2909
2910
2911
2912
2913
Cannot get：
2913
2914
2915
2916
Cannot get：
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
Cannot get：
2927
2928
2929
2930
2931
Cannot get：
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
Cannot get：
2940
2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
Cannot get：
2956
2957
2958
2959
Cannot get：
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
Cannot get：
2973
2974
2975
2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
298

4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
Cannot get：
4140
4141
4142
4143
4144
4145
Cannot get：
4145
4146
4147
Cannot get：
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158
4159
4160
4161
4162
4163
4164
4165
4166
4167
4168
4169
4170
4171
Cannot get：
4171
4172
Cannot get：
4172
4173
4174
4175
4176
4177
4178
4179
4180
4181
4182
Cannot get：
4182
4183
4184
4185
4186
4187
4188
Cannot get：
4188
4189
4190
4191
4192
4193
4194
Cannot get：
4194
4195
4196
Cannot get：
4196
4197
4198
Cannot get：
4198
4199
4200
4201
4202
4203
4204
4205
Cannot get：
4205
4206
4207
4208
4209
4210
4211
Cannot get：
4211
4212
4213
4214
4215
4216
4217
4218
4219
4220
4221
4222
4223
4224
4225
4226
4227
4228
4229
4230
Cannot get：
4230
4231
4232
4233
4234
4235
4236
4237
4238
4239
4240
4241
Cannot get：
4241
4242
Cannot get：
4242
4243
4244
Cannot get：
4244
4245
4246
4247
4248
4249
4250
4251
4252
4253
Cannot get：
4253
4254
4255
4256
4257
Cannot get：
4257

In [15]:
print(not_get_list)

[10, 33, 34, 41, 57, 59, 60, 66, 73, 79, 88, 94, 110, 115, 139, 151, 157, 159, 177, 200, 202, 228, 236, 240, 247, 266, 268, 272, 278, 284, 289, 303, 305, 310, 311, 322, 329, 330, 377, 380, 419, 468, 472, 476, 496, 508, 512, 513, 514, 517, 519, 520, 536, 545, 572, 583, 584, 588, 603, 612, 621, 648, 674, 675, 683, 689, 695, 700, 706, 716, 724, 725, 726, 769, 771, 773, 783, 784, 785, 786, 789, 791, 806, 818, 853, 892, 895, 924, 929, 936, 952, 958, 984, 1005, 1036, 1082, 1085, 1098, 1103, 1117, 1133, 1139, 1176, 1197, 1208, 1224, 1227, 1245, 1335, 1342, 1345, 1346, 1377, 1386, 1408, 1440, 1459, 1475, 1494, 1525, 1544, 1549, 1553, 1558, 1560, 1578, 1623, 1626, 1640, 1654, 1670, 1693, 1721, 1722, 1723, 1725, 1739, 1741, 1743, 1746, 1805, 1859, 1893, 1975, 1989, 2006, 2041, 2066, 2090, 2126, 2163, 2172, 2196, 2198, 2203, 2221, 2249, 2273, 2293, 2295, 2308, 2328, 2374, 2425, 2429, 2447, 2493, 2518, 2546, 2549, 2551, 2572, 2578, 2584, 2620, 2652, 2662, 2663, 2680, 2684, 2691, 2724, 2731, 2744, 

In [20]:
not_get_list5 = []
for i in not_get_list4: 
    ts_code = ts_list[i]
    try:
        df2 = pro.daily_basic(ts_code=ts_code, start_date=start_time, end_date=end_time, fields='ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv')
        df1 = df1.append(df2)
    except:
        time.sleep(1)
        print(i)
        not_get_list5.append(i)

In [21]:
len(df1)

8992027

In [22]:
df = reduce_mem_usage(df1, verbose=True)

Memory usage after optimization is: 926.15 MB
Decreased by 28.9%


In [23]:
df1.to_csv(raw_data_path + 'daily_full.csv', index=False)

In [25]:
# basic info内存优化
df = reduce_mem_usage(df1, verbose=True)

Memory usage after optimization is: 0.00 MB
Decreased by nan%


In [28]:

ts_list = data.ts_code.values
ts_len = data.ts_code.values.shape[0]
import time
not_get_list=[]
f = pro.limit_list
n = 0
for i in range(ts_len):
    print(i)
    ts_code = ts_list[i]
    try:
        if n == 0:
            df1 = f(ts_code=ts_code, start_date=start_time, end_date=end_time)
            n += 1
        else:
            df2 = f(ts_code=ts_code, start_date=start_time, end_date=end_time)
            df1 = df1.append(df2)
    except:
        time.sleep(1)
        print("Cannot get:")
        print(i)
        not_get_list.append(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
Cannot get:
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Cannot get:
100
101
Cannot get:
101
102
Cannot get:
102
103
Cannot get:
103
104
Cannot get:
104
105
Cannot get:
105
106
Cannot get:
106
107
Cannot get:
107
108
Cannot get:
108
109
Cannot get:
109
110
Cannot get:
110
111
Cannot get:
111
112
Cannot get:
112
113
Cannot get:
113
114
Cannot get:
114
115
Cannot get:
115
116
Cannot get:
116
117
Cannot get:
117
118
Cannot get:
118
119
Cannot get:
119
120
Cannot get:
120
121
Cannot get:
121
122
Cannot get:
122
123
Cannot get:
123
124
Cannot get:
124
125
Cannot get:
125
126
Cannot get:
126
127
Cannot get:
127
128
Cannot get:
128
129
Cannot get:
129
130
Cannot get:
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149

Cannot get:
1051
1052
Cannot get:
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
Cannot get:
1153
1154
Cannot get:
1154
1155
Cannot get:
1155
1156
Cannot get:
1156
1157
Cannot get:
1157
1158
Cannot get:
1158
1159
Cannot get:
1159
1160
Cannot get:
1160
1161
Cannot get:
1161
1162
Cannot get:
1162
1163
Cannot get:
1163
1164
Cannot get:
1164
1165
Cannot get:
1165
1166
Cannot get:
1166
1167
Cannot get:
1167
1168
Cannot get:
1168
1169
Cannot get:
1169
1170
Cannot get:
1170
1171
Cannot get:
1171
1172
Cannot get:
1172
1173
Cannot get:
1173

Cannot get:
1964
1965
Cannot get:
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
Cannot get:
2066
2067
Cannot get:
2067
2068
Cannot get:
2068
2069
Cannot get:
2069
2070
Cannot get:
2070
2071
Cannot get:
2071
2072
Cannot get:
2072
2073
Cannot get:
2073
2074
Cannot get:
2074
2075
Cannot get:
2075
2076
Cannot get:
2076
2077
Cannot get:
2077
2078
Cannot get:
2078
2079
Cannot get:
2079
2080
Cannot get:
2080
2081
Cannot get:
2081
2082
Cannot get:
2082
2083
Cannot get:
2083
2084
Cannot get:
2084
2085
Cannot get:
2085
2086
Cannot get:
2086

Cannot get:
2878
2879
2880
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
2974
2975
2976
2977
2978
2979
Cannot get:
2979
2980
Cannot get:
2980
2981
Cannot get:
2981
2982
Cannot get:
2982
2983
Cannot get:
2983
2984
Cannot get:
2984
2985
Cannot get:
2985
2986
Cannot get:
2986
2987
Cannot get:
2987
2988
Cannot get:
2988
2989
Cannot get:
2989
2990
Cannot get:
2990
2991
Cannot get:
2991
2992
Cannot get:
2992
2993
Cannot get:
2993
2994
Cannot get:
2994
2995
Cannot get:
2995
2996
Cannot get:
2996
2997
Cannot get:
2997
2998
Cannot get:
2998
2999
Cannot get:
2999
3000
Cannot get:
3000

3818
3819
3820
3821
3822
3823
3824
3825
3826
3827
3828
3829
3830
3831
3832
3833
3834
3835
3836
3837
3838
3839
3840
3841
3842
3843
3844
3845
3846
3847
3848
3849
3850
3851
3852
3853
3854
3855
3856
3857
3858
3859
3860
3861
3862
3863
3864
3865
3866
3867
3868
3869
3870
3871
3872
3873
3874
3875
3876
3877
3878
3879
3880
3881
3882
3883
3884
Cannot get:
3884
3885
Cannot get:
3885
3886
Cannot get:
3886
3887
Cannot get:
3887
3888
Cannot get:
3888
3889
Cannot get:
3889
3890
Cannot get:
3890
3891
Cannot get:
3891
3892
Cannot get:
3892
3893
Cannot get:
3893
3894
Cannot get:
3894
3895
Cannot get:
3895
3896
Cannot get:
3896
3897
Cannot get:
3897
3898
Cannot get:
3898
3899
Cannot get:
3899
3900
Cannot get:
3900
3901
Cannot get:
3901
3902
Cannot get:
3902
3903
Cannot get:
3903
3904
Cannot get:
3904
3905
Cannot get:
3905
3906
Cannot get:
3906
3907
Cannot get:
3907
3908
Cannot get:
3908
3909
Cannot get:
3909
3910
Cannot get:
3910
3911
Cannot get:
3911
3912
Cannot get:
3912
3913
Cannot get:
3913
3914
3915


In [30]:
print(not_get_list)

[28, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 627, 628, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 891, 892, 893, 894, 895, 896, 897, 898, 

In [33]:
not_get_list4 = []


In [35]:
print(not_get_list4)

[]


In [36]:
len(df1)

89399

In [37]:
df = reduce_mem_usage(df1, verbose=True)

Memory usage after optimization is: 6.99 MB
Decreased by 31.7%


In [38]:
df.to_csv(raw_data_path + 'price_limit.csv', index=False)

In [39]:
# 股东人数
pro = ts.pro_api()
df = pro.stk_holdernumber(ts_code='300199.SZ', start_date='20160101', end_date='20181231')

In [40]:
df

,ts_code,ann_date,end_date,holder_num
0,300199.SZ,20181025,20180930,25135
1,300199.SZ,20180808,20180630,25785
2,300199.SZ,20180426,20180331,23384
3,300199.SZ,20180316,20180228,23490
4,300199.SZ,20180316,20171231,24086
5,300199.SZ,20171026,20170930,24121
6,300199.SZ,20170817,20170630,26271
7,300199.SZ,20170427,20170331,24531
8,300199.SZ,20170427,20161231,22972
9,300199.SZ,20161028,20161027,19787


In [ ]:
# 股东人数统计
ts_list = data.ts_code.values
ts_len = data.ts_code.values.shape[0]
import time
not_get_list=[]
f = pro.cashflow
n = 0
for i in range(ts_len):
    print(i)
    ts_code = ts_list[i]
    try:
        if n == 0:
            df1 = f(ts_code=ts_code, start_date=start_time, end_date=end_time)
            n += 1
        else:
            df2 = f(ts_code=ts_code, start_date=start_time, end_date=end_time)
            df1 = df1.append(df2)
    except:
        time.sleep(1)
        print("Cannot get:")
        print(i)
        not_get_list.append(i)

0
1
2
3
4
5
6
7
8
9
Cannot get:
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
Cannot get:
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
Cannot get:
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


In [42]:
print(not_get_list)

[507, 585, 603, 656, 1272, 2387, 2456, 3454, 3999]


In [44]:
not_get_list2=[]
for i in not_get_list: 
    ts_code = ts_list[i]
    try:
        df2 = f(ts_code=ts_code, start_date=start_time, end_date=end_time)
        df1 = df1.append(df2)
    except:
        time.sleep(1)
        print(i)
        not_get_list2.append(i)
    

In [45]:
len(df1)

256861

In [50]:
df1

,ts_code,ann_date,end_date,holder_num
0,000001.SZ,20211021,20210930,390490.0
1,000001.SZ,20210820,20210630,296828.0
2,000001.SZ,20210421,20210331,331829.0
3,000001.SZ,20210202,20210201,290750.0
4,000001.SZ,20210202,20210129,290750.0
...,...,...,...,...
17,603878.SH,20171027,20170930,24308.0
18,603878.SH,20170830,20170630,24845.0
19,603878.SH,20170428,20170331,27337.0
20,603878.SH,20170414,20161231,43577.0


In [46]:
df = reduce_mem_usage(df1, verbose=True)

Memory usage after optimization is: 8.82 MB
Decreased by 10.0%


In [48]:
df.to_csv(raw_data_path+'shareholder_number.csv', index=False)

In [49]:
# 股东增减持样例
df = pro.stk_holdertrade(ts_code='600001.SZ')
df

,ts_code,ann_date,holder_name,holder_type,in_de,change_vol,change_ratio,after_share,after_ratio,avg_price,total_share


In [32]:
# 重新run一遍后复权数据
ts_list = data.ts_code.values
ts_len = data.ts_code.values.shape[0]
import time
not_get_list=[]
f = ts.pro_bar
n = 0
adj='hfq'
for i in range(ts_len):
    print(i)
    ts_code = ts_list[i]
    try:
        if n == 0:
            df1 = f(ts_code=i, start_date=start_time, adj=adj, end_date=end_time)
            n += 1
        else:
            df2 = f(ts_code=i, start_date=start_time, adj=adj, end_date=end_time)
            df1 = df1.append(df2)
    except:
        time.sleep(1)
        print("Cannot get:")
        print(i)
        not_get_list.append(i)
        

0
Cannot get:
0
1
Cannot get:
1
2
Cannot get:
2
3
Cannot get:
3
4
Cannot get:
4
5


KeyboardInterrupt: 